# TD Ameritrade Dashboard

#### Import Statements & Importing Variables from config.ipynb

In [ ]:
# from config import consumer_key, redirect_uri, json_path
# Jupyter Notebook Notation, pulling from 'config.ipynb'
%store -r consumer_key
%store -r redirect_uri
%store -r json_path
import pprint
from td.client import TDClient
import platform
import pandas as pd
import numpy as np

#### Assigned variable json_path from config.ipynb. The variable decides a file path based on operating system. I have two computers, one Mac and one PC. So the file paths are dependent on which computer I am on.

In [ ]:
json_path

#### Collecting API Token

In [ ]:
# Create a new instance of the client
td_client = TDClient(client_id = consumer_key, redirect_uri = redirect_uri, credentials_path = json_path)

In [ ]:
# Log in to a new session -- every 90 days -- comment out to avoid it autorunning with kernel restart
td_client.login()

#### Build dictionaries with values based on current account values

In [ ]:
# Get current quotes
# quotes_response = td_client.get_quotes(instruments = ['msft', 'aapl','sq','iipr'])
# quotes_response

In [ ]:
# Orders and Positions for account(s)
orders_and_positions = td_client.get_accounts(account = 'all', fields = ['orders', 'positions'])
# pprint.pprint(len(orders_and_positions))
orders_and_positions

In [ ]:
# # Collect all Orders and Positions for account(s)
# # Orders and Positions for account(s)
# orders_and_positions = td_client.get_accounts(account = 'all', fields = [])
# # pprint.pprint(len(orders_and_positions))
# orders_and_positions

#### Assigning account positions to proper lists in order to build out account DataFrames 

In [ ]:
stock_list = []
cash = []
for position in orders_and_positions[0]['securitiesAccount']['positions']:
    symbol = position['instrument']['symbol']
    if symbol != 'MMDA1':
        symbol = position['instrument']['symbol']
        quantity = position['longQuantity']
        search_instruments = td_client.search_instruments(symbol = symbol, projection = 'fundamental')
        quotes_response = td_client.get_quotes(instruments = [symbol])
        mark = quotes_response[symbol]['mark']
        beta = search_instruments[symbol]['fundamental']['beta']
#         print('+++++++++++++')
#         print('Market Value')
#         print(symbol + ' ' + '$' + str(quantity*mark) + ', ' + 'Beta of ' + str(beta))
        stock_list.append({'Symbol':symbol, 'Quantity':quantity, 'Mark':mark, 'Beta':beta})
        
    else:
        symbol = position['instrument']['symbol']
        quantity = position['longQuantity']
        mark = np.nan
        beta = np.nan
#         print('+++++++++++++')
#         print(symbol + ' ' + '$' + str(quantity))
        cash.append({'Symbol':symbol, 'Quantity':quantity, 'Mark':mark, 'Beta':beta})

#### Building out DataFrames with lists built above

In [ ]:
stocks = pd.DataFrame(stock_list)
beta_drop = pd.DataFrame(stock_list)
# Adding Value as a column to calculate the position value
stocks['Value'] = stocks['Quantity']*stocks['Mark']

cash = pd.DataFrame(cash)

portfolio = [stocks, cash]
portfolio = pd.concat(portfolio).reset_index().drop(['index'], axis = 1)
portfolio

#### Finding Beta values == 0, and replacing them with the mean beta calculated from all other Beta values excluding those positions with values equal to a Beta of 0

In [ ]:
drop = beta_drop[beta_drop['Beta']==0].index
beta_drop.drop(drop, inplace = True)
average_beta = beta_drop.Beta.mean()
stocks.loc[stocks.Beta == 0, 'Beta'] = average_beta
stocks

#### Taking a look at account values DataFrame

In [ ]:
stocks_value = round(sum(stocks['Value']),2)
cash_value = round(cash.iloc[0,1],2)
account_value = round((stocks_value + cash_value),2)

account_values = [{'Stocks':stocks_value, 'Cash':cash_value, 'Account Total': account_value, 'Account Beta': average_beta}]
account_values = pd.DataFrame(account_values)
account_values

In [ ]:
# search_instruments = td_client.search_instruments(symbol = 'IIPR', projection = 'fundamental')
# search_instruments

In [ ]:
# symbol_selection = orders_and_positions[0]['securitiesAccount']['positions'][0]['instrument']['symbol']
# quantity = orders_and_positions[0]['securitiesAccount']['positions'][0]['longQuantity']
# mark = quotes_response['IIPR']['mark']
# beta = search_instruments['IIPR']['fundamental']['beta']

# print('Market Value')
# print(symbol_selection + ' ' + '$' + str(quantity*mark) + ', ' + 'Beta of ' + str(beta))


In [ ]:
# Returns wether a market is open or close on certain days
# market_hours = td_client.get_market_hours(markets = ['BOND', 'EQUITY'], date = '2021-3-29')
# pprint.pprint(market_hours)